In [1]:
import numpy as np
import os

file_list = os.listdir('/content/SR250Breath')
windows = []
labels = []

for file_name in file_list:
    file_path = os.path.join('/content/SR250Breath', file_name)
    try:
        data = np.load(file_path)

        if file_path.endswith('.window.npy'):
            windows.append(data)
        elif file_path.endswith('.label.npy'):
            labels.append(data)
        else:
            print(f"Unknown file type: {file_path}")
    except Exception as e:
        print(f"Could not load {file_name}: {e}")

# You can access the arrays using the file names as keys, for example:
# print(data_arrays['Millenials_E_breath_sitting_desk_20250909-115853.window.npy'])

In [2]:
windows = np.concat(windows)
labels = np.concat(labels)

print(windows.shape)
print(labels.shape)

(36833, 11, 3, 120)
(36833,)


In [3]:
# in maniera tale che l'ultimo asse corrisponde alle antenne
windows = windows.transpose(0, 1, 3, 2)
print(windows.shape)

(36833, 11, 120, 3)


In [4]:
abs_windows = np.abs(windows)
print(abs_windows.shape)

(36833, 11, 120, 3)


In [5]:
phase_windows = np.angle(windows)

In [6]:
all = np.concatenate((abs_windows, phase_windows), axis=-1)
# all = phase_windows
print(all.shape)

(36833, 11, 120, 6)


In [7]:
import tensorflow as tf

# Assuming 'labels' is your numpy array of labels
# You might need to adjust the 'depth' parameter based on the number of unique classes in your labels
one_hot_labels = tf.one_hot(labels, depth=2) # Assuming 2 classes for demonstration

print(one_hot_labels.shape)

(36833, 2)


In [8]:
from sklearn.model_selection import train_test_split
import numpy as np

# Convert TensorFlow tensor to NumPy array
one_hot_labels_np = one_hot_labels.numpy()

# Split into training and temporary sets (for test and validation)
X_train, X_temp, y_train, y_temp = train_test_split(phase_windows, one_hot_labels_np, test_size=0.3, random_state=42)

# Split the temporary set into test and validation sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training set shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("\nValidation set shapes:")
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)
print("\nTest set shapes:")
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

Training set shapes:
X_train: (25783, 11, 120, 3)
y_train: (25783, 2)

Validation set shapes:
X_val: (5525, 11, 120, 3)
y_val: (5525, 2)

Test set shapes:
X_test: (5525, 11, 120, 3)
y_test: (5525, 2)


# Task
Produce a CNN model for the prediction of the label and train it using the training, test, and validation data split from the `abs_windows` and `labels` variables.

## Define the cnn model architecture

### Subtask:
Define the layers of the CNN model, including convolutional layers, pooling layers, and dense layers, suitable for your data shape.


**Reasoning**:
Define the CNN model architecture using convolutional, pooling, and dense layers.



**Reasoning**:
The error indicates that the input shape is too small for the convolutional and pooling layers. Adjust the kernel size and pooling size to be smaller to avoid negative dimensions.



In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(16, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(16, (1, 1), activation='relu'))
model.add(MaxPooling2D((2, 4)))

# Flatten the output
model.add(Flatten())

# Add dense layers
# model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_little = model

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 11, 120, 16)    │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 5, 60, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 4, 59, 16)      │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 2, 29, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 2, 29, 16)      │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 7, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           226 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,602 (6.26 KB)

 Trainable params: 1,602 (6.26 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(MaxPooling2D((2, 4)))

# Flatten the output
model.add(Flatten())

# Add dense layers
# model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_small = model

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 11, 120, 32)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 60, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 4, 59, 64)      │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 2, 29, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 2, 29, 128)     │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,794 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,498 (72.26 KB)

 Trainable params: 18,498 (72.26 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (2, 2), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((1, 2))) # Pool only along the second dimension
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))
model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))


# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_medium = model

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 10, 119, 32)    │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 10, 59, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 9, 58, 64)      │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 9, 29, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 8, 28, 128)     │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 8, 14, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 14336)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     1,835,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,876,962 (7.16 MB)

 Trainable params: 1,876,962 (7.16 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with further adjusted kernel and pooling sizes
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((1, 1))) # Use (1,1) pooling to avoid reducing dimension too quickly
model.add(Conv2D(64, (1, 1), activation='relu'))
model.add(MaxPooling2D((1, 1)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(MaxPooling2D((1, 1)))


# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_large = model

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 11, 120, 32)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 11, 120, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 11, 120, 64)    │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 11, 120, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 11, 120, 128)   │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 11, 120, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 168960)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    21,627,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,637,826 (82.54 MB)

 Trainable params: 21,637,826 (82.54 MB)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
The model summary shows that the output shape after the last MaxPooling2D layer has a dimension of 0, which is incorrect. The input shape is likely still too large for the kernel and pooling sizes. I need to further adjust the kernel and pooling sizes to ensure the output dimensions remain positive.



In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (1, 1), activation='relu'))
model.add(MaxPooling2D((2, 4)))


# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_small_test1 = model

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 11, 120, 32)    │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 5, 60, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 4, 59, 64)      │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 2, 29, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 2, 29, 128)     │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 1, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 896)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       114,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,778 (514.76 KB)

 Trainable params: 131,778 (514.76 KB)

 Non-trainable params: 0 (0.00 B)

**Reasoning**:
The model summary now shows positive dimensions after the pooling layers, but using (1,1) kernel and pooling sizes essentially bypasses the spatial feature extraction capabilities of CNNs. I need to find a balance between kernel/pooling sizes and the input shape to effectively use convolutional layers without causing negative dimensions. I will try increasing the kernel size slightly while keeping the pooling size small.



In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Add convolutional layers with adjusted kernel and pooling sizes
model.add(Conv2D(32, (2, 2), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D((1, 2))) # Pool only along the second dimension
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))
model.add(Conv2D(128, (2, 2), activation='relu'))
model.add(MaxPooling2D((1, 2)))


# Flatten the output
model.add(Flatten())

# Add dense layers
# model.add(Dense(128, activation='relu'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.summary()

model_small_test2 = model

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 10, 119, 32)    │           416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 10, 59, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 9, 58, 64)      │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 9, 29, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 8, 28, 128)     │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 8, 14, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 14336)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │        28,674 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,242 (274.38 KB)

 Trainable params: 70,242 (274.38 KB)

 Non-trainable params: 0 (0.00 B)

## Compile the model

### Subtask:
Compile the CNN model by specifying the optimizer, loss function, and metrics.


**Reasoning**:
Compile the CNN model with the Adam optimizer, binary crossentropy loss, and accuracy metric.



In [15]:
from tensorflow.keras.optimizers import Adam

model = model_small_test2

model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Train the model

### Subtask:
Train the compiled model using the training data (`X_train`, `y_train`) and validate it using the validation data (`X_val`, `y_val`).


**Reasoning**:
Train the compiled model using the training and validation data.



In [16]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - accuracy: 0.8308 - loss: 0.4593 - val_accuracy: 0.8387 - val_loss: 0.4294
Epoch 2/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8315 - loss: 0.4218 - val_accuracy: 0.8547 - val_loss: 0.3833
Epoch 3/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8851 - loss: 0.2844 - val_accuracy: 0.8972 - val_loss: 0.2671
Epoch 4/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9560 - loss: 0.1306 - val_accuracy: 0.9102 - val_loss: 0.2321
Epoch 5/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9791 - loss: 0.0627 - val_accuracy: 0.9207 - val_loss: 0.2335
Epoch 6/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9905 - loss: 0.0342 - val_accuracy: 0.9258 - val_loss: 0.2342
Epoch 7/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9890 - loss: 0.0310 - val_accuracy: 0.9265 - val_loss: 0.2256
Epoch 8/10
806/806 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.9922 - loss: 0.0241 - val_accuracy: 0

**Reasoning**:
The first step is to load the data from the CSV file into a pandas DataFrame and display the first few rows to understand its structure.



## Evaluate the model

### Subtask:
Evaluate the trained model on the test data (`X_test`, `y_test`) to assess its performance.

**Reasoning**:
Evaluate the trained model on the test data to assess its performance.

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

173/173 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9291 - loss: 0.2918
Test Loss: 0.2926829755306244
Test Accuracy: 0.9281448125839233


In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Get predictions from the model
y_pred_prob = model.predict(X_test)

# Convert probabilities to class predictions (assuming binary classification and sigmoid activation)
# Since y_test is one-hot encoded, y_pred should also be one-hot encoded for comparison with some metrics
# However, precision_score, recall_score, and f1_score with average='binary' expect binary labels.
# Let's convert both to binary labels for these metrics.
y_pred = (y_pred_prob[:, 1] > 0.5).astype(int) # Assuming the second column is the positive class probability

# Convert one-hot encoded true labels back to single class labels
y_true = np.argmax(y_test, axis=1)

# Calculate metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

173/173 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Precision: 0.8500749625187406
Recall: 0.6524741081703107
F1 Score: 0.73828125


In [19]:
import tensorflow as tf

# Quantize the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save the quantized model (optional)
# with open('quantized_model.tflite', 'wb') as f:
#     f.write(quantized_tflite_model)

# Load the quantized model
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare test data for inference
# TFLite models expect float32 inputs
X_test_float32 = X_test.astype(np.float32)

# Run inference on the test set
predictions = []
for i in range(X_test_float32.shape[0]):
    interpreter.set_tensor(input_details[0]['index'], np.expand_dims(X_test_float32[i], axis=0))
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predictions.append(output_data[0])

predictions = np.array(predictions)

# Evaluate the quantized model (assuming y_test is already one-hot encoded)
# For binary classification with sigmoid output, you can use binary_accuracy
from tensorflow.keras.metrics import BinaryAccuracy

binary_accuracy = BinaryAccuracy()
binary_accuracy.update_state(y_test, predictions)

print(f"Quantized Model Test Accuracy: {binary_accuracy.result().numpy()}")

# If you want to calculate loss, you'd need to implement it manually or convert predictions to match y_test format
# For example, if y_test is one-hot, and predictions are probabilities for each class:
# from tensorflow.keras.losses import BinaryCrossentropy
# bce = BinaryCrossentropy()
# loss = bce(y_test, predictions).numpy()
# print(f"Quantized Model Test Loss: {loss}")

Saved artifact at '/tmp/tmpk50dz3vx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 11, 120, 3), dtype=tf.float32, name='keras_tensor_212')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  137673435832080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137673435836304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137673435837648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137673435836880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137673435836496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137673435838416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137673435838608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137673435837840: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Quantized Model Test Accuracy: 0.9279637932777405


In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# predictions from the quantized model were already calculated in cell SUFu4NnP0VZO

# Convert probabilities to class predictions (assuming binary classification and sigmoid activation)
y_pred_quantized = (predictions[:, 1] > 0.5).astype(int) # Assuming the second column is the positive class probability

# Convert one-hot encoded true labels back to single class labels
y_true = np.argmax(y_test, axis=1)

# Calculate metrics for the quantized model
precision_quantized = precision_score(y_true, y_pred_quantized)
recall_quantized = recall_score(y_true, y_pred_quantized)
f1_quantized = f1_score(y_true, y_pred_quantized)

print(f"Quantized Model Precision: {precision_quantized}")
print(f"Quantized Model Recall: {recall_quantized}")
print(f"Quantized Model F1 Score: {f1_quantized}")

Quantized Model Precision: 0.8513513513513513
Quantized Model Recall: 0.6524741081703107
Quantized Model F1 Score: 0.7387622149837133


In [21]:
os.mkdir('/content/models')

model.save(os.path.join('/content/models', 'model_small_test2' + '2.keras'))

In [22]:
# Float model export:
# The second argument, string format, must be the model name + 2.keras
model_small_test2 = tf.keras.models.load_model(os.path.join('/content/models', 'model_small_test22.keras'))

converter = tf.lite.TFLiteConverter.from_keras_model(model_small_test2)
tflite_model = converter.convert()
print("Float model size:", open(os.path.join('/content/models', 'model_small_test2' + '.tflite'), "wb").write(tflite_model))

Saved artifact at '/tmp/tmpiwyehb7p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 11, 120, 3), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  137672196360912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196364368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196365136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196366288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196363984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196363792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196364560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196359184: TensorSpec(shape=(), dtype=tf.resource, name=None)
Float model size: 284592


In [23]:
print(X_train[0].shape)

(11, 120, 3)


In [24]:
# Quantized model export:

# Definition of Representative Dataset generator:
def representative_data_gen():
  for i in range(X_train.shape[0]):
    yield [X_train[i].reshape((-1,) + X_train[i].shape)]

#def representative_dataset():
#  for i in range(100):
#    yield [ np.array([(np.random.rand(1960)).astype(np.float32)]) ]

converter = tf.lite.TFLiteConverter.from_keras_model(model_small_test2)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.compat.v1.lite.constants.INT8 # or tf.uint8
converter.inference_output_type = tf.compat.v1.lite.constants.INT8  # or tf.uint8

tflite_model_quant = converter.convert()
print("Quantized model size: ", open(os.path.join('/content/models', 'model_small_test2' + '-int8.tflite'), "wb").write(tflite_model_quant))

Saved artifact at '/tmp/tmp5f57cjkb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 11, 120, 3), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  137672196360912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196364368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196365136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196366288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196363984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196363792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196364560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137672196359184: TensorSpec(shape=(), dtype=tf.resource, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Quantized model size:  80480


In [25]:
!apt-get update && apt-get -qq install xxd

MODEL_TFLITE = '/content/models/'+ 'model_small_test2' +'-int8.tflite'
MODEL_TFLITE_MICRO = 'TinyConvModel-int8.cc'

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,305 kB]
Get:14 http://archive.u

In [26]:
!xxd -i {MODEL_TFLITE} > {MODEL_TFLITE_MICRO}